In [50]:
import dask
import re
import dask
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import datetime

#air
#reserve
df_air_reserve=pd.read_csv('/Users/aleric/Desktop/kaggle/all (2)/air_reserve.csv')
df_air_reserve['visit_date']=df_air_reserve['visit_datetime'].str.split().str[0]
df_air_reserve['visit_time']=df_air_reserve['visit_datetime'].str.split().str[1]
df_air_reserve['reserve_date']=df_air_reserve['reserve_datetime'].str.split().str[0]
df_air_reserve['reserve_time']=df_air_reserve['reserve_datetime'].str.split().str[1]

# group by store_id & visit_date
df_air_reserve_grouped=df_air_reserve.groupby(['air_store_id','visit_date']).sum()
df_air_reserve_grouped=df_air_reserve_grouped.reset_index()

#store info
df_air_store_info=pd.read_csv('/Users/aleric/Desktop/kaggle/all (2)/air_store_info.csv')

#visit
df_air_visit_data=pd.read_csv('/Users/aleric/Desktop/kaggle/all (2)/air_visit_data.csv')

#date info
df_date_info=pd.read_csv('/Users/aleric/Desktop/kaggle/all (2)/date_info.csv')

In [51]:
# join tables df_air_reserve_grouped, df_air_store_info & df_air_visit_data
df_air=df_air_reserve_grouped.merge(df_air_store_info, left_on='air_store_id', right_on='air_store_id', how='outer')
df_air=df_air.merge(df_air_visit_data, left_on=['air_store_id', 'visit_date'], right_on=['air_store_id', 'visit_date'], how='outer')

# join date_info
df_air=df_air.merge(df_date_info, left_on=['visit_date'], right_on=['calendar_date'], how='left')

In [52]:

df_air.drop(['calendar_date'], axis=1,inplace=True)
df_air.head()


air_store_id  visit_date  reserve_visitors  air_genre_name  \
0  air_00a91d42b08b08d9  2016-10-31               2.0  Italian/French   
1  air_00a91d42b08b08d9  2016-12-05               9.0  Italian/French   
2  air_00a91d42b08b08d9  2016-12-14              18.0  Italian/French   
3  air_00a91d42b08b08d9  2016-12-17               2.0  Italian/French   
4  air_00a91d42b08b08d9  2016-12-20               4.0  Italian/French   

                     air_area_name   latitude   longitude  visitors  \
0  Tōkyō-to Chiyoda-ku Kudanminami  35.694003  139.753595      26.0   
1  Tōkyō-to Chiyoda-ku Kudanminami  35.694003  139.753595      33.0   
2  Tōkyō-to Chiyoda-ku Kudanminami  35.694003  139.753595      30.0   
3  Tōkyō-to Chiyoda-ku Kudanminami  35.694003  139.753595      17.0   
4  Tōkyō-to Chiyoda-ku Kudanminami  35.694003  139.753595      37.0   

  day_of_week  holiday_flg  
0      Monday          0.0  
1      Monday          0.0  
2   Wednesday          0.0  
3    Saturday          0.0  
4     Tuesday          0.0

In [53]:


#hpg
# reserve & visit data
df_hpg_reserve=pd.read_csv('/Users/aleric/Desktop/kaggle/all (2)/hpg_reserve.csv')
df_hpg_reserve['visit_date']=df_hpg_reserve['visit_datetime'].str.split().str[0]
df_hpg_reserve['visit_time']=df_hpg_reserve['visit_datetime'].str.split().str[1]
df_hpg_reserve['reserve_date']=df_hpg_reserve['reserve_datetime'].str.split().str[0]
df_hpg_reserve['reserve_time']=df_hpg_reserve['reserve_datetime'].str.split().str[1]
# group by store_id & visit_date
df_hpg_reserve_grouped=df_hpg_reserve.groupby(['hpg_store_id', 'visit_date']).sum()
df_hpg_reserve_grouped=df_hpg_reserve_grouped.reset_index()

#store info
df_hpg_store_info=pd.read_csv('/Users/aleric/Desktop/kaggle/all (2)/hpg_store_info.csv')

# join tables df_hpg_reserve & df_hpg_store_info
df_hpg=df_hpg_reserve_grouped.merge(df_hpg_store_info, on='hpg_store_id', how='outer')

# join air & hpg
df_store_id_relation=pd.read_csv('/Users/aleric/Desktop/kaggle/all (2)/store_id_relation.csv')
df=df_air.merge(df_store_id_relation, on='air_store_id', how='left')
df=df.merge(df_hpg, on=['hpg_store_id', 'visit_date'], how='left')

# drop unecessary columns
df.drop(['reserve_visitors_x', 'reserve_visitors_y', 'latitude_x', 'latitude_y', 'longitude_x', 'longitude_y'],axis=1, inplace=True)
df.drop(['hpg_store_id', 'hpg_genre_name', 'hpg_area_name'],axis=1, inplace=True)

# change date format to datetime
df['visit_date']=pd.to_datetime(df['visit_date'])

In [54]:
# encoding the day_of_week to catagorical type
dow=pd.get_dummies(df.day_of_week)
df_merge_dow=pd.concat([df,dow],axis="columns")
df_merge_dow



air_store_id visit_date  air_genre_name  \
0       air_00a91d42b08b08d9 2016-10-31  Italian/French   
1       air_00a91d42b08b08d9 2016-12-05  Italian/French   
2       air_00a91d42b08b08d9 2016-12-14  Italian/French   
3       air_00a91d42b08b08d9 2016-12-17  Italian/French   
4       air_00a91d42b08b08d9 2016-12-20  Italian/French   
5       air_00a91d42b08b08d9 2017-02-18  Italian/French   
6       air_00a91d42b08b08d9 2017-02-23  Italian/French   
7       air_00a91d42b08b08d9 2017-03-01  Italian/French   
8       air_00a91d42b08b08d9 2017-03-14  Italian/French   
9       air_00a91d42b08b08d9 2017-03-21  Italian/French   
10      air_00a91d42b08b08d9 2017-03-24  Italian/French   
11      air_00a91d42b08b08d9 2017-04-04  Italian/French   
12      air_0164b9927d20bcc3 2016-10-28  Italian/French   
13      air_0164b9927d20bcc3 2016-11-01  Italian/French   
14      air_0164b9927d20bcc3 2016-11-02  Italian/French   
15      air_0164b9927d20bcc3 2016-11-08  Italian/French   
16      air_0164b9927d20bcc3 2016-11-10  Italian/French   
17      air_0164b9927d20bcc3 2016-11-11  Italian/French   
18      air_0164b9927d20bcc3 2016-11-14  Italian/French   
19      air_0164b9927d20bcc3 2016-11-15  Italian/French   
20      air_0164b9927d20bcc3 2016-11-16  Italian/French   
21      air_0164b9927d20bcc3 2016-11-18  Italian/French   
22      air_0164b9927d20bcc3 2016-11-21  Italian/French   
23      air_0164b9927d20bcc3 2016-11-22  Italian/French   
24      air_0164b9927d20bcc3 2016-11-24  Italian/French   
25      air_0164b9927d20bcc3 2016-11-26  Italian/French   
26      air_0164b9927d20bcc3 2016-11-28  Italian/French   
27      air_0164b9927d20bcc3 2016-12-02  Italian/French   
28      air_0164b9927d20bcc3 2016-12-05  Italian/French   
29      air_0164b9927d20bcc3 2016-12-06  Italian/French   
...                      ...        ...             ...   
254359  air_24e8414b9b07decb 2017-03-17             NaN   
254360  air_24e8414b9b07decb 2017-03-18             NaN   
254361  air_24e8414b9b07decb 2017-03-21             NaN   
254362  air_24e8414b9b07decb 2017-03-22             NaN   
254363  air_24e8414b9b07decb 2017-03-23             NaN   
254364  air_24e8414b9b07decb 2017-03-24             NaN   
254365  air_24e8414b9b07decb 2017-03-25             NaN   
254366  air_24e8414b9b07decb 2017-03-27             NaN   
254367  air_24e8414b9b07decb 2017-03-28             NaN   
254368  air_24e8414b9b07decb 2017-03-29             NaN   
254369  air_24e8414b9b07decb 2017-03-30             NaN   
254370  air_24e8414b9b07decb 2017-03-31             NaN   
254371  air_24e8414b9b07decb 2017-04-01             NaN   
254372  air_24e8414b9b07decb 2017-04-03             NaN   
254373  air_24e8414b9b07decb 2017-04-04             NaN   
254374  air_24e8414b9b07decb 2017-04-05             NaN   
254375  air_24e8414b9b07decb 2017-04-06             NaN   
254376  air_24e8414b9b07decb 2017-04-07             NaN   
254377  air_24e8414b9b07decb 2017-04-08             NaN   
254378  air_24e8414b9b07decb 2017-04-10             NaN   
254379  air_24e8414b9b07decb 2017-04-11             NaN   
254380  air_24e8414b9b07decb 2017-04-12             NaN   
254381  air_24e8414b9b07decb 2017-04-13             NaN   
254382  air_24e8414b9b07decb 2017-04-14             NaN   
254383  air_24e8414b9b07decb 2017-04-15             NaN   
254384  air_24e8414b9b07decb 2017-04-18             NaN   
254385  air_24e8414b9b07decb 2017-04-19             NaN   
254386  air_24e8414b9b07decb 2017-04-20             NaN   
254387  air_24e8414b9b07decb 2017-04-21             NaN   
254388  air_24e8414b9b07decb 2017-04-22             NaN   

                          air_area_name  visitors day_of_week  holiday_flg  \
0       Tōkyō-to Chiyoda-ku Kudanminami      26.0      Monday          0.0   
1       Tōkyō-to Chiyoda-ku Kudanminami      33.0      Monday          0.0   
2       Tōkyō-to Chiyoda-ku Kudanminami      30.0   Wednesday          0.0   
3       Tōkyō-to Chiyoda-ku Kudanminami      

In [55]:
# divide a month into 3 parts(early,middle,late)
# early:1~10 of a month
# middle:11~20 a month
# late:else
df_merge_dow["day"]=df_merge_dow["visit_date"].dt.day
df_merge_dow["month_3division"]=df_merge_dow["day"].apply(
    lambda x:"early_of_month" if x<=10 else( "middle_of_month" if 10<x and x<=20 else "late_of_month"))

month_3division=pd.get_dummies(df_merge_dow.month_3division)
df_merge_dow_month=pd.concat([df_merge_dow,month_3division],axis="columns")
# air_area_name,air_genre_name is deleted as well since model will be made to each unique air_store_id
df_merge_dow_month.drop(["air_area_name","air_genre_name","day","month_3division","day_of_week"],axis=1, inplace=True)
df_merge_dow_month


air_store_id visit_date  visitors  holiday_flg  Friday  \
0       air_00a91d42b08b08d9 2016-10-31      26.0          0.0       0   
1       air_00a91d42b08b08d9 2016-12-05      33.0          0.0       0   
2       air_00a91d42b08b08d9 2016-12-14      30.0          0.0       0   
3       air_00a91d42b08b08d9 2016-12-17      17.0          0.0       0   
4       air_00a91d42b08b08d9 2016-12-20      37.0          0.0       0   
5       air_00a91d42b08b08d9 2017-02-18      18.0          0.0       0   
6       air_00a91d42b08b08d9 2017-02-23      40.0          0.0       0   
7       air_00a91d42b08b08d9 2017-03-01      17.0          0.0       0   
8       air_00a91d42b08b08d9 2017-03-14      28.0          0.0       0   
9       air_00a91d42b08b08d9 2017-03-21      28.0          0.0       0   
10      air_00a91d42b08b08d9 2017-03-24      34.0          0.0       1   
11      air_00a91d42b08b08d9 2017-04-04      17.0          0.0       0   
12      air_0164b9927d20bcc3 2016-10-28      25.0          0.0       1   
13      air_0164b9927d20bcc3 2016-11-01      17.0          0.0       0   
14      air_0164b9927d20bcc3 2016-11-02      10.0          0.0       0   
15      air_0164b9927d20bcc3 2016-11-08       9.0          0.0       0   
16      air_0164b9927d20bcc3 2016-11-10       7.0          0.0       0   
17      air_0164b9927d20bcc3 2016-11-11       3.0          0.0       1   
18      air_0164b9927d20bcc3 2016-11-14      19.0          0.0       0   
19      air_0164b9927d20bcc3 2016-11-15       7.0          0.0       0   
20      air_0164b9927d20bcc3 2016-11-16       8.0          0.0       0   
21      air_0164b9927d20bcc3 2016-11-18      18.0          0.0       1   
22      air_0164b9927d20bcc3 2016-11-21       3.0          0.0       0   
23      air_0164b9927d20bcc3 2016-11-22      10.0          0.0       0   
24      air_0164b9927d20bcc3 2016-11-24       2.0          0.0       0   
25      air_0164b9927d20bcc3 2016-11-26      11.0          0.0       0   
26      air_0164b9927d20bcc3 2016-11-28       6.0          0.0       0   
27      air_0164b9927d20bcc3 2016-12-02      25.0          0.0       1   
28      air_0164b9927d20bcc3 2016-12-05       6.0          0.0       0   
29      air_0164b9927d20bcc3 2016-12-06       3.0          0.0       0   
...                      ...        ...       ...          ...     ...   
254359  air_24e8414b9b07decb 2017-03-17       4.0          0.0       1   
254360  air_24e8414b9b07decb 2017-03-18       7.0          0.0       0   
254361  air_24e8414b9b07decb 2017-03-21       2.0          0.0       0   
254362  air_24e8414b9b07decb 2017-03-22       8.0          0.0       0   
254363  air_24e8414b9b07decb 2017-03-23       8.0          0.0       0   
254364  air_24e8414b9b07decb 2017-03-24       7.0          0.0       1   
254365  air_24e8414b9b07decb 2017-03-25       2.0          0.0       0   
254366  air_24e8414b9b07decb 2017-03-27       4.0          0.0       0   
254367  air_24e8414b9b07decb 2017-03-28       7.0          0.0       0   
254368  air_24e8414b9b07decb 2017-03-29      11.0          0.0       0   
254369  air_24e8414b9b07decb 2017-03-30       7.0          0.0       0   
254370  air_24e8414b9b07decb 2017-03-31       8.0          0.0       1   
254371  air_24e8414b9b07decb 2017-04-01       6.0          0.0       0   
254372  air_24e8414b9b07decb 2017-04-03       4.0          0.0       0   
254373  air_24e8414b9b07decb 2017-04-04       2.0          0.0       0   
254374  air_24e8414b9b07decb 2017-04-05       2.0          0.0       0   
254375  air_24e8414b9b07decb 2017-04-06       8.0          0.0       0   
254376  air_24e8414b9b07decb 2017-04-07       8.0          0.0       1   
254377  air_24e8414b9b07decb 2017-04-08       8.0          0.0       0   
254378  air_24e8414b9b07decb 2017-04-10       4.0          0.0       0   
254379  air_24e8414b9b07decb 2017-04-11       2.0          0.0       0   
254380  air_24e8414b9b07decb 2017-04-12       8.0          0.0       0   
254381  air_24e8414b

In [56]:
df_merge_dow_month.to_csv("/Users/aleric/Desktop/kaggle/all (2)/input_for_modle_holiday&dow&3division_month.csv")